**Check Usage of GPUs**

In [ ]:
!nvidia-smi

## 0. Imports

In [ ]:
import utils

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.utils import save_image

import PIL
from pathlib import Path

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import copy
import random
import time

import sys
import os
import requests

sys.path.append('./mae')
!pip3 install timm==0.4.5  # 0.3.2 does not work in Colab

import models_mae
import mlp

**Set CUDA Device**

In [ ]:
torch.cuda.set_device(2)

In [ ]:
torch.cuda.current_device()

## 1. Create CLS_DATASET Dataset Class

In [ ]:
class CLS_DATASET(Dataset):

    def __init__(self, root, cls_dir, csv_file, transform=None):
        self.root = root
        self.cls_dir = cls_dir
        self.cls_files = [f for f in os.listdir(cls_dir) if f.endswith('.pt')]
        self.cls_files = sorted(self.cls_files)
    
        self.data_frame = pd.read_csv(csv_file)
        self.data = torch.LongTensor(self.data_frame.values.tolist())
        self.data = self.data.squeeze()
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # load each load_batch_size items
        load_batch_size = 50 
        cls_idx = index//load_batch_size
        cls_vec_name = os.path.join(self.cls_dir, self.cls_files[cls_idx])
        cls_vec = torch.load(cls_vec_name)
        cls = cls_vec[index%load_batch_size,:].unsqueeze(0)

        label = self.data[index]
        if self.transform:
            cls = self.transform(cls)
        return (cls, label)

In [ ]:
## CHOOSE DATASET
# dataset = 'CIFAR10'
dataset = 'CIFAR100'
# dataset = 'RESISC45'
# dataset = 'AID'

In [ ]:
# SEED = 1234
SEED = 123

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# mask_ratio = 0.3
# mask_ratio = 0.1
mask_ratio = 0.5

ROOT = '/data/ek58_data/'
train_cls_dir = ROOT+'cls_tokens_/'+dataset+'/train_'+str(mask_ratio)+'/data'
test_cls_dir = ROOT+'cls_tokens_/'+dataset+'/test_'+str(mask_ratio)+'/data'
train_csv_file = ROOT+'cls_tokens_/'+dataset+'/train_'+str(mask_ratio)+'/train_cls_y.csv'
test_csv_file = ROOT+'cls_tokens_/'+dataset+'/test_'+str(mask_ratio)+'/test_cls_y.csv'

train_data = CLS_DATASET(root=ROOT,
                           cls_dir=train_cls_dir,
                           csv_file=train_csv_file)

test_data = CLS_DATASET(root=ROOT,
                           cls_dir=test_cls_dir,
                           csv_file=test_csv_file)

if dataset == 'CIFAR10':
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
elif dataset == 'CIFAR100':
    classes = range(100)
elif dataset == 'RESISC45':
    classes = range(45)
elif dataset == 'AID':
    classes = range(30)
    
BATCH_SIZE = 64
# BATCH_SIZE = 16
# BATCH_SIZE = 1

train_iterator = data.DataLoader(train_data,
#                                  shuffle=False,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

test_iterator = data.DataLoader(test_data,
                                batch_size=BATCH_SIZE)


## 2. Training the Model

In [ ]:
input_dim = train_data[0][0].numel()
output_dim = len(classes)

hidden_in = 1000
# hidden_out = 500
hidden_out = 1000

model = mlp.MLP(input_dim, output_dim, hidden_in, hidden_out)

In [ ]:
model

In [ ]:
optimizer = optim.Adam(model.parameters())
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
EPOCHS = 30

train_loss_vec = []; train_acc_vec = []
test_loss_vec = []; test_acc_vec = []
for epoch in trange(EPOCHS):
    start_time = time.monotonic()

    train_loss, train_acc = mlp.train(model, train_iterator, optimizer, criterion, device)
    test_loss, test_acc = mlp.evaluate(model, test_iterator, criterion, device)

    train_loss_vec.append(train_loss); train_acc_vec.append(train_acc)
    test_loss_vec.append(test_loss); test_acc_vec.append(test_acc)

    end_time = time.monotonic()

    epoch_mins, epoch_secs = utils.epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tTest Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

## 3. Examining the Model

In [ ]:
model_str = 'cls_MLP_' + dataset + '_fixedmask_' + str(mask_ratio) + '_ep_' + str(EPOCHS)

### ***Save Model Logs***

In [ ]:
np.savetxt('logs/fixedmask/' + str(mask_ratio) + '/' + model_str + '_train_loss.log', train_loss_vec, fmt='%1.4f')
np.savetxt('logs/fixedmask/' + str(mask_ratio) + '/' + model_str + '_test_loss.log', test_loss_vec, fmt='%1.4f')
np.savetxt('logs/fixedmask/' + str(mask_ratio) + '/' + model_str + '_train_acc.log', train_acc_vec, fmt='%1.4f')
np.savetxt('logs/fixedmask/' + str(mask_ratio) + '/' + model_str + '_test_acc.log', test_acc_vec, fmt='%1.4f')